### Introduction

This notebook is intended to train a Decision Transformer using offline data gathered from exploring the CarRacing-v2 environment with a pre-trained DQN model.

### Install initial environment in Google Colab

In [1]:
import sys
import os

if 'google.colab' in sys.modules:
  if not os.path.exists('/content/.already_installed'):
    !git clone https://github.com/YakivGalkin/cnn_decision_transformer
    !apt-get install -y swig
    !pip install -r cnn_decision_transformer/requirements.txt
    with open('/content/.already_installed', 'w') as f:
        f.write('done')
  %cd /content/cnn_decision_transformer

Cloning into 'cnn_decision_transformer'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 53 (delta 19), reused 39 (delta 8), pack-reused 0
Receiving objects: 100% (53/53), 1.28 MiB | 24.31 MiB/s, done.
Resolving deltas: 100% (19/19), done.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  swig4.0
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  swig swig4.0
0 upgraded, 2 newly installed, 0 to remove and 19 not upgraded.
Need to get 1,116 kB of archives.
After this operation, 5,542 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig4.0 amd64 4.0.2-1ubuntu1 [1,110 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig all 4.0.2-1ubuntu1 [5,632 B]
F

### Load Dataset

In [2]:
#car_racing_15_100
#offline_car_racing_150_1000

import utils.storage as storage
features = storage.load_dataset('offline_car_racing_150_1000')
print(len(features["observations"]))

Download complete.
150


In [3]:
import gymnasium as gym
env =  gym.make('CarRacing-v2', continuous=False) #, render_mode='human'

In [8]:
from dataclasses import asdict, dataclass
import wandb
import os


@dataclass
class TrainConfig:
    # WANDB CONFIG
    wandb_id: str = "dt_25"
    wandb_name: str = "DT_25"
    model_save_name: str = "DT_MODEL_25"
    saved_model_version: str = "latest"
    save_steps: int = 10000

    # TRAINING DATA CONFIG
    num_train_epochs: int = 100
    max_ep_len: int = 1000
    max_length: int = 10
    rtg_gamma: float = 1.0

    prefix: str = 'DT'
    log_interval: int = 5
    save_steps: int = 30
    per_device_train_batch_size: int = 1
    learning_rate: float = 0.0001
    weight_decay: float = 0.0001
    warmup_ratio: float = 0.1
    max_grad_norm: float = 0.25


trainConfig = TrainConfig()

os.environ["WANDB_DISABLED"] = "false"
os.environ['WANDB_NOTEBOOK_NAME'] = 'DT_train.ipynb'
os.environ["WANDB_LOG_MODEL"] = "checkpoint"

wandb.login(key="f060d3284088ffaf4624e2de8b236f39711a99a2") # move to .env!
wandb.init(resume=trainConfig.wandb_id,
           name = trainConfig.wandb_name,
           mode="online",
           entity="yakiv",
            project="CarRacingDT",
            #resume= "allow"
            config=asdict(trainConfig)
           )


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


train/epoch,▁▁▂▂▂▂▃▃▃▃▃▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇███▁▂▂▂▂▂▃▁▁▁▂
train/global_step,▁▁▂▂▂▂▃▃▃▃▃▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇███▁▂▂▂▂▂▃▁▁▁▂
train/learning_rate,▁▁▂▂▂▂▃▃▃▃▃▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇███▁▂▂▂▂▂▃▁▁▁▂
train/loss,▇█▆▆▆▅▅▄▅▅▄▄▄▅▃▃▄▃▂▂▂▃▂▂▂▃▁▂▂▇▆▅▆▅▄▄█▇█▆
train/epoch,0.8
train/global_step,120
train/learning_rate,1e-05
train/loss,1.532


### Train

In [9]:

from visual_decision_transformer.visual_decision_transformer_trainable import VisualDecisionTransformerGymDataCollator, TrainableVisualDecisionTransformer
from visual_decision_transformer.configuration import DecisionTransformerConfig
from utils.dataset_wrappers import DummyDataset
from utils.dataset_wrappers import CarRacingFeatureDataset
from transformers import Trainer, TrainingArguments

feature_dataset = CarRacingFeatureDataset(src=features)
collator = VisualDecisionTransformerGymDataCollator(feature_dataset, max_len=trainConfig.max_length,   max_ep_len=trainConfig.max_ep_len,)

dt_config = DecisionTransformerConfig(state_dim=collator.state_dim, act_dim=collator.act_dim,
                                      max_length = trainConfig.max_length,
                                      max_ep_len = trainConfig.max_ep_len,
                                      )

model = TrainableVisualDecisionTransformer(dt_config)


training_args = TrainingArguments(
    output_dir="output/",
    report_to="wandb",
    save_steps= trainConfig.save_steps,
    remove_unused_columns=False,
    optim="adamw_torch",
    num_train_epochs=trainConfig.num_train_epochs,
    per_device_train_batch_size= trainConfig.per_device_train_batch_size,
    learning_rate= trainConfig.learning_rate,
    weight_decay= trainConfig.weight_decay,
    warmup_ratio= trainConfig.warmup_ratio,
    max_grad_norm= trainConfig.max_grad_norm,
    logging_steps= trainConfig.log_interval,
)



trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=DummyDataset(len(feature_dataset)), #there is a 'hack'  - no need to pass actual data
    data_collator=collator,

)

trainer.train()


loss: 1.5263149738311768


Step,Training Loss
5,1.518300
10,1.673100
15,1.663800
20,1.569800
25,1.578100
30,1.662000
35,1.555000
40,1.578900
45,1.576700
50,1.597700


loss: 1.4841265678405762
loss: 1.4726547002792358
loss: 1.430295705795288
loss: 1.6781971454620361
loss: 1.692381501197815
loss: 1.7963730096817017
loss: 1.69246506690979
loss: 1.5979727506637573
loss: 1.5861690044403076
loss: 1.6122392416000366
loss: 1.7260528802871704
loss: 1.578924298286438
loss: 1.8102591037750244
loss: 1.591295599937439
loss: 1.5294163227081299
loss: 1.6025301218032837
loss: 1.5057637691497803
loss: 1.5245697498321533
loss: 1.6867077350616455
loss: 1.6493905782699585
loss: 1.599858283996582
loss: 1.5805222988128662
loss: 1.6042280197143555
loss: 1.4566867351531982
loss: 1.626922607421875
loss: 1.7308788299560547
loss: 1.7103157043457031
loss: 1.5793192386627197
loss: 1.6627591848373413


wandb: Adding directory to artifact (./output/checkpoint-30)... Done. 0.1s


loss: 1.5696275234222412
loss: 1.4377272129058838
loss: 1.5644044876098633
loss: 1.624131441116333
loss: 1.5789417028427124
loss: 1.6229280233383179
loss: 1.5636639595031738
loss: 1.5959123373031616
loss: 1.6173051595687866
loss: 1.4949276447296143
loss: 1.369848370552063
loss: 1.5550800561904907
loss: 1.5491259098052979
loss: 1.7963275909423828
loss: 1.613073706626892
loss: 1.6293083429336548
loss: 1.4958410263061523
loss: 1.6783708333969116
loss: 1.6510636806488037
loss: 1.533854365348816
loss: 1.6599557399749756
loss: 1.5881978273391724
loss: 1.5249254703521729
loss: 1.485162615776062
loss: 1.7259899377822876
loss: 1.5857763290405273
loss: 1.6361608505249023
loss: 1.5330660343170166
loss: 1.588605284690857
loss: 1.6586637496948242


...

wandb: Adding directory to artifact (./output/checkpoint-900)... Done. 0.1s


loss: 0.7063612341880798
loss: 1.374328851699829
loss: 1.076373815536499
loss: 0.9027762413024902
loss: 0.38892102241516113
loss: 0.8857256174087524
loss: 1.3693790435791016
loss: 0.8955772519111633
loss: 1.075865387916565
loss: 0.9252927899360657
loss: 0.2615390717983246
loss: 0.7410931587219238
loss: 0.7875353693962097
loss: 0.852269172668457
loss: 1.3718106746673584
loss: 1.4287455081939697
loss: 0.6574356555938721
loss: 1.4120427370071411
loss: 0.4901684820652008
loss: 0.8935451507568359
loss: 1.0536129474639893
loss: 0.6427475214004517
loss: 0.5678585171699524
loss: 0.6042290925979614
loss: 0.6905237436294556
loss: 1.0135974884033203
loss: 1.529565691947937
loss: 1.085613489151001
loss: 1.482613444328308
loss: 0.39598268270492554


wandb: Adding directory to artifact (./output/checkpoint-930)... Done. 0.1s


loss: 0.8567681312561035
loss: 0.5602818727493286
loss: 1.259456992149353
loss: 0.3017203211784363
loss: 0.7938013672828674
loss: 0.7035360336303711
loss: 0.8280556797981262
loss: 1.7363885641098022
loss: 0.7760562896728516
loss: 0.8345940709114075
loss: 1.127901315689087
skipping batch item


ValueError: ignored

In [ ]:
#play
import matplotlib.pyplot as plt
from IPython.display import display as ipy_display, clear_output
#import gymnasium as gym
# build the environment
max_ep_len = 1000
device = 'cpu'
model = model.to('cpu')
scale = 1000.0  # normalization for rewards/returns
TARGET_RETURN = 900 / scale  # evaluation is conditioned on a return of 12000, scaled accordingly

env =  gym.make('CarRacing-v2', render_mode='rgb_array', continuous=False) #,

state_dim = 96*96*3
act_dim = 1
# Create the decision transformer model

# Interact with the environment and create a video
episode_return, episode_length = 0, 0
[state, _] = env.reset()
state = prepare_observation_array(state)
target_return = torch.tensor(TARGET_RETURN, device=device, dtype=torch.float32).reshape(1, 1)
states = torch.from_numpy(state).reshape(1, state_dim).to(device=device, dtype=torch.float32)
actions = torch.zeros((0, act_dim),  device=device, dtype=torch.long)
rewards = torch.zeros(0, device=device, dtype=torch.float32)
print_every = 10
iter = 0

timesteps = torch.tensor(0, device=device, dtype=torch.long).reshape(1, 1)
for t in range(max_ep_len):
    iter += 1
    actions = torch.cat([actions, torch.zeros((1, act_dim), dtype=torch.long,  device=device)], dim=0)
    rewards = torch.cat([rewards, torch.zeros(1, device=device)])

    action = get_action(
        model,
        states,
        actions,
        rewards,
        target_return,
        timesteps,
    )

    action =   torch.argmax(action).item() # action.detach().cpu().numpy()

    actions[-1] = torch.tensor(action, dtype=torch.long)

    state, reward, done, _, _ = env.step(action)

    if iter % print_every ==0:
      image = env.render()
      clear_output(wait=True)
      plt.imshow(image)
      plt.axis('off')  # Hide the axis
      display(plt.gcf())



    state = prepare_observation_array(state)
    cur_state = torch.from_numpy(state).to(device=device).reshape(1, state_dim)
    states = torch.cat([states, cur_state], dim=0)
    rewards[-1] = reward

    pred_return = target_return[0, -1] - (reward / scale)
    target_return = torch.cat([target_return, pred_return.reshape(1, 1)], dim=1)
    timesteps = torch.cat([timesteps, torch.ones((1, 1), device=device, dtype=torch.long) * (t + 1)], dim=1)

    episode_return += reward
    episode_length += 1

    if done:
        break